We need:


1.   Document Collection (https://github.com/federicarollo/Italian-Crime-News)
2.   Test Queries
3.   Prejudged Assessments on the Queries






We have to
*   Build a parser - Ula
*   Build a preprocessor - Viktor
*   Build the data structures (lexicon, docIndex,statistics,directIndex,inverseIndex) - Mateusz
*   Implement a Ranking Strategy - Ala
*   Find qrels
*   Evaluate








In [ ]:
# !pip install -q python-terrier

In [1]:
from tqdm.auto import tqdm
import pyterrier as pt
import numpy as np
import nltk
import string
from collections import Counter, defaultdict
import re
import pickle
import os, humanize
import time

In [2]:
nltk.download("stopwords", quiet=True)
stopwords = set(nltk.corpus.stopwords.words("english"))
stemmer = nltk.stem.PorterStemmer().stem

acronym_re = re.compile(r"\.(?!(\S))")
punctuation_wo_dot = string.punctuation.replace(".", "")
trans_table = str.maketrans(
    "‘’´“”–-" + punctuation_wo_dot, "'''\"\"--" + " " * len(punctuation_wo_dot)
)


def preprocess(s):
    # lowercasing
    s = s.lower()
    # ampersand
    s = s.replace("&", " and ")
    # acronyms
    s = acronym_re.sub("", s)
    # remove punctuation and handle special chars
    s = s.translate(trans_table)
    # tokeniser
    s = s.split()
    # stopwords
    s = filter(lambda t: t not in stopwords, s)
    # stemming
    return list(map(stemmer, s))

In [3]:
dataset = pt.get_dataset("msmarco_passage")
queries = dataset.get_topics("test-2020")
qrels = dataset.get_qrels("test-2020")

dataset.get_corpus()

Java started (triggered by _read_topics_singleline) and loaded: pyterrier.java, pyterrier.terrier.java [version=5.10 (build: craigm 2024-08-22 17:33), helper_version=0.0.8]


['C:\\Users\\USER/.pyterrier\\corpora\\msmarco_passage\\corpus\\collection.tsv']

In [4]:
DOCS = 0
for _ in dataset.get_corpus_iter():
    DOCS += 1

In [5]:
def defaultlexicon():
    return [0, 0]


def build_index(docs):

    lexicon = defaultdict(defaultlexicon)  # token -> df, tf
    docids = defaultdict(list)  # token -> list of docids
    freqs = defaultdict(list)  # token -> list of frequencies in each doc
    doc_index = {}  # docno -> size of document, length normalization component

    doc_len_sum = 0

    for doc in tqdm(docs.get_corpus_iter(), total=DOCS):
        docid = doc["docno"]
        tokens = preprocess(doc["text"])
        token_tf = Counter(tokens)
        doc_index[docid] = (len(tokens), 0)
        doc_len_sum += len(tokens)

        for token, tf in token_tf.items():
            lexicon[token][0] += 1
            lexicon[token][1] += tf
            docids[token].append(docid)
            freqs[token].append(tf)

    stats = {
        "num_docs": int(docid),
        "num_terms": len(lexicon),
        "avg_doc_len": doc_len_sum / DOCS,
    }
    return {
        "lexicon": lexicon,
        "docids": docids,
        "freqs": freqs,
        "doc_index": doc_index,
        "stats": stats,
    }

In [6]:
if not os.path.exists("index.pickle"):
    index = build_index(dataset)
    # update length normalisation components in doc_index
    b = 0.75
    for docid, (doc_len, _) in tqdm(index["doc_index"].items(), total=DOCS):
        index["doc_index"][docid] = (
            doc_len,
            (1 - b) + b * (doc_len / index["stats"]["avg_doc_len"]),
        )
    with open("index.pickle", "wb") as f:
        pickle.dump(index, f)
    print(
        f'The index requires {humanize.naturalsize(os.path.getsize("index.pickle"))} bytes'
    )
else:
    with open("index.pickle", "rb") as f:
        index = pickle.load(f)
    print(f"Loaded index with {index['stats']['num_docs']} documents")

  0%|          | 0/8841823 [00:00<?, ?it/s]

  0%|          | 0/8841823 [00:00<?, ?it/s]

The index requires 1.9 GB bytes


In [7]:
#  BM25 ranking function
def bm25_ranking(query, index: dict, k=1.5, b=0.75):
    query_tokens = preprocess(query)
    scores = defaultdict(float)

    for token in query_tokens:
        if token in index["lexicon"]:
            doc_freq = len(index["docids"][token])
            for doc_id, tf in zip(index["docids"][token], index["freqs"][token]):
                idf = np.log(index["stats"]["num_docs"] / doc_freq)
                numerator = tf
                denominator = tf + k * index["doc_index"][doc_id][1]
                bm25_score = idf * (numerator / denominator)

                scores[doc_id] += bm25_score

    ranked_docs = sorted(scores.items(), key=lambda item: item[1], reverse=True)
    return ranked_docs

In [8]:
def ndcg_at_k(r, k):
    def dcg_at_k(r, k):
        r = np.asarray(r)[:k]
        return sum(r[i] / max(1, np.log2(i + 1)) for i in range(0, k))

    idcg = dcg_at_k(sorted(r, reverse=True), k)
    if not idcg:
        return 0.0
    return dcg_at_k(r, k) / idcg

In [9]:
def ap_at_k(r, k):
    r = np.asarray(r)[:k] != 0
    N = sum(r)
    if N == 0:
        return 0
    return sum([sum(r[: i + 1]) / (i + 1) * r[i] for i in range(k)]) / N

In [10]:
def benchmark(timing=True, evaluate=True):
    ndcg_at_10 = []
    aps_at_10 = []

    art = 0  # average ranking time

    for i, query in tqdm(queries.iterrows(), total=len(queries)):
        start = time.time()
        results = bm25_ranking(query["query"], index)
        if timing:
            end = time.time()
            art = (i * art + end - start) / (i + 1)
            print(f"Ranking time:\t\t{end-start:.5f}")
            print(f"Average ranking time:\t{art:.5f}")

        if evaluate:
            rel_qrels = qrels[qrels["qid"] == query["qid"]].set_index("docno")
            relevance_judgments = [
                rel_qrels["label"].get(docno, 0) for docno, _ in results
            ]

            ndcg_at_10.append(ndcg_at_k(relevance_judgments, 10))
            aps_at_10.append(ap_at_k(relevance_judgments, 10))
            print("")
            print(f"nDCG@10:\t\t{ndcg_at_10[-1]:.5f}")
            print(f"AP@10:\t\t\t{aps_at_10[-1]:.5f}")
        print("----------------------------------------------------------")
    if evaluate:
        print(f"Mean nDCG @ 10:\t\t\t{ np.mean(ndcg_at_10):.5f}")
        print(f"Mean Average Precision @ 10:\t{np.mean(aps_at_10):.5f}")

In [11]:
benchmark(timing=True, evaluate=False)

  0%|          | 0/200 [00:00<?, ?it/s]

Ranking time:		0.01200
Average ranking time:	0.01200
----------------------------------------------------------
Ranking time:		0.01900
Average ranking time:	0.01550
----------------------------------------------------------
Ranking time:		0.24781
Average ranking time:	0.09294
----------------------------------------------------------
Ranking time:		0.10300
Average ranking time:	0.09545
----------------------------------------------------------
Ranking time:		4.42046
Average ranking time:	0.96045
----------------------------------------------------------
Ranking time:		0.41500
Average ranking time:	0.86955
----------------------------------------------------------
Ranking time:		0.85951
Average ranking time:	0.86811
----------------------------------------------------------
Ranking time:		0.16600
Average ranking time:	0.78035
----------------------------------------------------------
Ranking time:		0.98475
Average ranking time:	0.80306
---------------------------------------------------